# Dissecting createPhosimCatalog

### Motivation 

A notebook to understand how does createPhosimCatalog work in detail, including all the dependencies on `ts_wep`, `ts_ofc`  and `ts_phosim`.  We need to be able to describe the catalog creation to prepare the `star_separation_test.ipynb`.

## Setup 

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import os
import argparse
import numpy as np

from lsst.ts.wep.ParamReader import ParamReader
from lsst.ts.wep.Utility import FilterType

from lsst.ts.ofc.Utility import InstName
from lsst.ts.ofc.ctrlIntf.OFCCalculationFactory import OFCCalculationFactory

from lsst.ts.phosim.SkySim import SkySim
from lsst.ts.phosim.OpdMetrology import OpdMetrology
from lsst.ts.phosim.Utility import getAoclcOutputPath, getConfigDir


In [29]:
getAoclcOutputPath()

'/astro/store/epyc/projects/lsst_comm/aoclc_output/'

The code below is from ../analysis_scripts/createPhosimCatalog.py

In [7]:
class createPhosimCatalog():

    def createPhosimCatalog(self, numStars, starSep, magList,
                            raOffset, decOffset, outputFilePath):

        """
        numStars: number of stars per field

        starSep: Minimum separation between stars

        magList: Star magnitudes in catalog

        outputFilePath: Filename for output catalog
        """

        # Survey parameters
        surveySettingFilePath = os.path.join(getConfigDir(),
                                            "surveySettings.yaml")
        surveySettings = ParamReader(filePath=surveySettingFilePath)
        filterType = FilterType.fromString(
            surveySettings.getSetting("filterType"))
        raInDeg = surveySettings.getSetting("raInDeg")
        decInDeg = surveySettings.getSetting("decInDeg")
        rotAngInDeg = surveySettings.getSetting("rotAngInDeg")

        ofcCalc = self._prepareOfcCalc(filterType, rotAngInDeg)
        skySim = SkySim()
        metr = OpdMetrology()
        metr.setDefaultComcamGQ()

        skySim = self._addStarsInField(skySim, metr, numStars,
                                       starSep, raOffset, decOffset,
                                       magList)
        skySim.exportSkyToFile(outputFilePath)


    def _prepareOfcCalc(self, filterType, rotAngInDeg):

        ofcCalc = OFCCalculationFactory.getCalculator(InstName.COMCAM)
        ofcCalc.setFilter(filterType)
        ofcCalc.setRotAng(rotAngInDeg)
        ofcCalc.setGainByPSSN()

        return ofcCalc


    def _addStarsInField(self, skySim, opdMetr, numStars, starSep,
                         raOffset, decOffset, starMagList):

        starId = 0
        raInDegList, declInDegList = opdMetr.getFieldXY()

        raShift = np.zeros(numStars)
        decMin = -1 * starSep * (float(numStars-1) / 2)
        decMax = -1 * decMin
        decShift = np.linspace(decMin, decMax, numStars)

        raShift += raOffset
        decShift += decOffset

        for raInDeg, declInDeg in zip(raInDegList, declInDegList):
            # It is noted that the field position might be < 0. But it is not the
            # same case for ra (0 <= ra <= 360).
            # if (raInDeg < 0):
            #     raInDeg += 360.0

            for num in range(numStars):
                raPerturbed = raInDeg + raShift[num]
                decPerturbed = declInDeg + decShift[num]

                if raPerturbed < 0:
                    raPerturbed += 360.0

                skySim.addStarByRaDecInDeg(starId, raPerturbed,
                                           decPerturbed, starMagList[num])
                starId += 1

        return skySim





And it is called within createPhosimCatalog.py as :

In [ ]:
if __name__ == "__main__":

    # Set the parser
    parser = argparse.ArgumentParser(
        description="Create a Star Catalog for comcamCloseLoop simulation.")
    parser.add_argument("--numStars", type=int, default=1,
                        help="number of stars per field (default: 1)")
    parser.add_argument("--raOffset", type=float, default=0.0,
                        help="Offset from center of chip in RA in arcsec (default: 0.)")
    parser.add_argument("--decOffset", type=float, default=0.0,
                        help="Offset from center of chip in dec in arcsec (default: 0.)")
    parser.add_argument("--starSep", type=float, default=0.05,
                        help="minimum separation between stars in degrees (default: 0.05)")
    parser.add_argument("--magFaintLim", type=float, default=15.,
                        help="faint limit of stars in catalog (default: 15)")
    parser.add_argument("--magBrightLim", type=float, default=15.,
                        help="bright limit of stars in catalog (default: 15)")
    parser.add_argument("--output", type=str, default="",
                        help="output catalog filepath")
    args = parser.parse_args()

    if (args.output == ""):
        outputDir = getAoclcOutputPath()
        outputFilePath = os.path.join(outputDir, "starCat.txt")
    else:
        outputFilePath = args.output

    starMagList = np.linspace(args.magBrightLim, args.magFaintLim, args.numStars)

    createCat = createPhosimCatalog()
    createCat.createPhosimCatalog(args.numStars, args.starSep,
                                  args.magFaintLim, args.magBrightLim,
                                  args.raOffset, args.decOffset,
                                  outputFilePath)

An example usage of createPhosimCatalog.py was in runSnrAnalysis.py, as : 

In [8]:
magVal = 17.5
createCat = createPhosimCatalog()
raShiftAsec = 2
decShiftAsec =       2
skyFilePath = 'starCat.txt'
raShift = (raShiftAsec * .2) / 3600 # Convert to degrees
decShift = (decShiftAsec * .2) / 3600 # Convert to degrees
createCat.createPhosimCatalog(1, 0, [magVal], raShift, decShift, skyFilePath)




This dissects the createPhosimCatalog()  class 

In [57]:
# this includes the following routines : 

# see https://docs.python.org/3.3/tutorial/classes.html 

# class createPhosimCatalog(self, numStars, starSep, magList,
#                             raOffset, decOffset, outputFilePath):

#         """
#         numStars: number of stars per field

#         starSep: Minimum separation between stars

#         magList: Star magnitudes in catalog

#         outputFilePath: Filename for output catalog
#         """


numStars = 1
starSep = 0
magList = [magVal]
raOffset = raShift # this defines the offset of each star from the CCD center ... 
decOffset = decShift 
outputFilePath = skyFilePath 

# Survey parameters
surveySettingFilePath = os.path.join(getConfigDir(),"surveySettings.yaml")
    #  getConfigDir() returns '/astro/store/epyc/projects/lsst_comm/ts_phosim/policy'  ,
    # so surveySettingFilePath = '/astro/store/epyc/projects/lsst_comm/ts_phosim/policy/surveySettings.yaml'
    # i.e. tells the parameter reader where the settings are 
    # the file surveySettings.yaml contains :
    
'''
# Survey Settings for Catalogs

# Filter Type
filterType: ref

# RA in degrees
raInDeg: 0.0

# Dec in degrees
decInDeg: 0.0

# Rotation Angle in degrees
rotAngInDeg: 0.0

'''
surveySettings = ParamReader(filePath=surveySettingFilePath)
    #  ParamReader() is a parameters reader for yaml format class
    # in /astro/store/epyc/projects/lsst_comm/ts_wep/python/lsst/ts/wep/ParamReader.py 
    # this means that it can return a value of each parameter as a method 
    # below surveySettings.getSetting() queries for a value of each paramater
    # in the surveySettings.yaml file 
    # so eg. surveySettings.getSetting("filterType") = 'ref'
filterType = FilterType.fromString(surveySettings.getSetting("filterType"))
    # FilterType is an enumeration, i.e. a mapping of string filter name to numbers: 
'''
U = 1
G = 2
R = 3
I = 4
Z = 5
Y = 6
REF = 7
'''

# so filterType = 7 (float)
 
raInDeg = surveySettings.getSetting("raInDeg")  # so raInDeg = 0 
decInDeg = surveySettings.getSetting("decInDeg") # decInDeg = 0 
rotAngInDeg = surveySettings.getSetting("rotAngInDeg")  # rotAngInDeg = 0 


# This is a private method :
# ofcCalc = self._prepareOfcCalc(filterType, rotAngInDeg)
# _prepareOfcCalc()  contains 
ofcCalc = OFCCalculationFactory.getCalculator(InstName.COMCAM)
# here InstName is another enumeration :it changes 
'''
LSST = 1
COMCAM = 2
SH = 3
CMOS = 4
'''
# So InstName.COMCAM  yields 2 
# the following settings apply to the Optical Feedback Control calculator, that 
# processes the wavefront error 
ofcCalc.setFilter(filterType)
ofcCalc.setRotAng(rotAngInDeg)
ofcCalc.setGainByPSSN()


# this instantiates  the sky simulation SkySim() class, which lives in 
# https://github.com/lsst-ts/ts_phosim/blob/master/python/lsst/ts/phosim/SkySim.py
skySim = SkySim()

# this instantiates the Optical Path Difference metrology class, which lives in 
# https://github.com/lsst-ts/ts_phosim/blob/master/python/lsst/ts/phosim/OpdMetrology.py 
metr = OpdMetrology()
# the following sets the default comcam Gaussian Quadrature field X, Y and weighting ratio
# ComCam is the central raft of LSST cam, which is composed of 3 x 3 CCDs
metr.setDefaultComcamGQ()


# now this redefines skySim , using another private method : 
#skySim = self._addStarsInField(skySim, metr, numStars,
#                               starSep, raOffset, decOffset,
#                               magList)

# _addStarsInField(self, skySim, opdMetr, numStars, starSep,
#                         raOffset, decOffset, starMagList):




In [58]:
opdMetr = metr 
starMagList = magList 
starId = 0
# for ComCam, there are #3x3 CCDs, i.e. 3 rows and 3 cols . That is 9 fields total.
# getFieldXY() method gives the XY coordinates of the center of each CCD in degrees 

raInDegList, declInDegList = opdMetr.getFieldXY()
# (array([-0.2347, -0.2347, -0.2347,  0.    ,  0.    ,  0.    ,  0.2347,
#         0.2347,  0.2347]),
# array([-0.2347,  0.    ,  0.2347, -0.2347,  0.    ,  0.2347, -0.2347,
#         0.    ,  0.2347]))

raShift = np.zeros(numStars)
    
    
decMin = -1 * starSep * (float(numStars-1) / 2)
decMax = -1 * decMin
decShift = np.linspace(decMin, decMax, numStars)


In [68]:
raShift += raOffset
decShift += decOffset

# this iterates over the x,y positions of each CCD center 
# in degrees 
# it adds numStars to EACH  CCD,  and the starCatalog lists 
# all stars, so if we choose one star per CCD, it will return a list of nine stars 

for raInDeg, declInDeg in zip(raInDegList, declInDegList):
    # It is noted that the field position might be < 0. But it is not the
    # same case for ra (0 <= ra <= 360).
    # if (raInDeg < 0):
    #     raInDeg += 360.0

    for num in range(numStars):
        raPerturbed = raInDeg + raShift[num]
        decPerturbed = declInDeg + decShift[num]

        if raPerturbed < 0:
            raPerturbed += 360.0

        skySim.addStarByRaDecInDeg(starId, raPerturbed,
                                   decPerturbed, starMagList[num])
        starId += 1

#return skySim

# these methods could print the content of star catalog : 
# skySim.getRaDecInDeg()
# skySim.getStarMag()
# skySim.getStarId()

# this method exports the list of stars for the 9 ComCam CCDs to a file 
skySim.exportSkyToFile(outputFilePath)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

So to make a catalog that includes 2 stars with a certain separation  starSep (in degrees), do : 

In [9]:
magVal = 17.5
createCat = createPhosimCatalog()
raShiftAsec = 2
decShiftAsec = 2
skyFilePath = 'starCat.txt'
raShift = (raShiftAsec * .2) / 3600 # Convert to degrees
decShift = (decShiftAsec * .2) / 3600 # Convert to degrees

createCat.createPhosimCatalog(2, 0.05, [magVal,magVal], raShift, decShift, skyFilePath)



In [15]:
np.arange(0.01, 0.2347, 0.025 )

array([0.01 , 0.035, 0.06 , 0.085, 0.11 , 0.135, 0.16 , 0.185, 0.21 ])

In [16]:
from lsst.ts.phosim.PhosimCmpt import PhosimCmpt

So the `numPro` parameter refers to the number of processors in the phosim component setting file. We can check what it is by inspecting the config directory : 

In [21]:
from lsst.ts.phosim.Utility import getPhoSimPath, getAoclcOutputPath, \
                                   getConfigDir

In [30]:
getConfigDir()

'/astro/store/epyc/projects/lsst_comm/ts_phosim/policy'

In [31]:
phosimCmpt.getIntraFocalDirName()

NameError: name 'phosimCmpt' is not defined

In [24]:
!ls /astro/store/epyc/projects/lsst_comm/ts_phosim/policy


M1M3  camera   instFile		       surveySettings.yaml
M2    cmdFile  phosimCmptSetting.yaml  teleSetting.yaml


In [26]:
!more /astro/store/epyc/projects/lsst_comm/ts_phosim/policy/phosimCmptSetting.yaml

---

# PhoSim component setting

# Number of zk (annular Zernike polynomials) terms
numOfZk: 19

# Intra-focal directory name. This is needed if the defocal image is by the
# camera piston.
intraDirName: intra

# Extra-focal directory name. This is needed if the defocal image is by the
# camera piston.
extraDirName: extra

# PhoSim-related default parameters

# Number of processors. The value should be >= 1.
numPro: 1

# Whether to generate amplifier images (1 = true, 0 = false)
e2ADC: 1
